## Domácí úloha do předmětu NI-MPI

`Ondřej Schejbal - schejond`

In [1]:
# sada použitých hodnot pro gammu a sada použitých metod
# hodnoty pro gammu je možné libovolně přidávat
gammas = [3., 2., 1.]
methods = ['Jacobi', 'SOR']

In [2]:
import numpy as np

* Pro zadanou gammu se připraví matice A, vektor b, počáteční vektor x_0 a dále matice D a L
* Jednotlivé matice a vektory jsou zkontruovány podle dimenzí a tvaru, které odpovídá zadání, resp. handoutu předmětu

In [3]:
def prepareVariables(gamma):
    A = np.eye(20, dtype=np.double)*gamma - np.eye(20, k=1, dtype=np.double) - np.eye(20, k=-1, dtype=np.double)
    
    b = np.ones((20,1), dtype=np.double) * (gamma - 2)
    b[0] = gamma - 1
    b[-1] = gamma - 1

    x_0 = np.zeros((20,1), dtype=np.double)
    
    D = np.eye(20, dtype=np.double) * A.diagonal()

    L = np.tril(A, -1)
    
    return A, b, x_0, D, L

* Při výpočtech jsem se rozhodl použít eukleidův vzorec pro výpočet normy

In [4]:
def getResidueValue(x_k, A, b):
    return np.matmul(A, x_k) - b

In [5]:
def getRelativeAccuracy(r_k, b):
    return np.divide(np.linalg.norm(r_k), np.linalg.norm(b))

In [6]:
# funkce používaná ke kontrole podmínky z věty 32.2
def matrixSolvable(Q, A):
    E = np.zeros((20, 20), dtype=np.double)
    np.fill_diagonal(E, 1)
    W = E - np.matmul(np.linalg.inv(Q), A)
    maxValue = -1
    for eigenvalue in np.linalg.eig(W)[0]:
        if maxValue < abs(eigenvalue):
            maxValue = abs(eigenvalue)
    return maxValue < 1

In [7]:
def printResult(iterationCount, finalPrecision, x_result, gamma, method):
    print('Pro gammu =', gamma, 'a metodu', method, 'je požadovaná přesnost splněna po', iterationCount,
         'iteracích. Nalezený vektor má přesnost:', str(finalPrecision) + '.')
    print('Nalezený vektor:', x_result)

In [8]:
# obecná rekurzivní metoda, která postupně updatuje hodnotu x_i dokud není splněna požadovaná přesnost ( < 10^-6) 
def findAccurateX(A, b, x_0, Q, gamma, methodName):
    i = 0
    x_i = x_0
    while not getRelativeAccuracy(getResidueValue(x_i, A, b), b) < np.double(10e-6):
        x_i = np.matmul(np.linalg.inv(Q), (np.matmul((Q - A), x_i) + b))
        i += 1

    printResult(i, getRelativeAccuracy(getResidueValue(x_i, A, b), b), x_i, gamma, methodName)

In [9]:
# hledání hodnot x pro jednotlivé hodnoty parametru gamma za pomocí jednotlivých metod
for method in methods:
    for gamma in gammas:
        A, b, x_0, D, L = prepareVariables(gamma)
        
        if method is 'Jacobi':
            Q = D
        elif method is 'SOR':
            Q = D + L
        else:
            print('Neznámý název metody!')
            break

        # check if we can iteratively find the solution
        if not matrixSolvable(Q, A):
            print('Není řešitelné pomocí', method, 'metody pro gammu =', str(gamma) +
                  ', protože diverguje. Spektrální poloměr totiž není < 1. Viz věta 32.2\n')
            continue
        findAccurateX(A, b, x_0, Q, gamma, method)
        print()

Pro gammu = 3.0 a metodu Jacobi je požadovaná přesnost splněna po 28 iteracích. Nalezený vektor má přesnost: 7.078347037507666e-06.
Nalezený vektor: [[0.99999825]
 [0.99999661]
 [0.99999498]
 [0.99999365]
 [0.99999234]
 [0.99999142]
 [0.99999053]
 [0.99999001]
 [0.99998957]
 [0.99998941]
 [0.99998941]
 [0.99998957]
 [0.99999001]
 [0.99999053]
 [0.99999142]
 [0.99999234]
 [0.99999365]
 [0.99999498]
 [0.99999661]
 [0.99999825]]

Pro gammu = 2.0 a metodu Jacobi je požadovaná přesnost splněna po 782 iteracích. Nalezený vektor má přesnost: 9.968534524306131e-06.
Nalezený vektor: [[0.99997097]
 [0.99994259]
 [0.9999155 ]
 [0.99989029]
 [0.99986753]
 [0.99984773]
 [0.99983133]
 [0.9998187 ]
 [0.99981012]
 [0.99980578]
 [0.99980578]
 [0.99981012]
 [0.9998187 ]
 [0.99983133]
 [0.99984773]
 [0.99986753]
 [0.99989029]
 [0.9999155 ]
 [0.99994259]
 [0.99997097]]

Není řešitelné pomocí Jacobi metody pro gammu = 1.0, protože diverguje. Spektrální poloměr totiž není < 1. Viz věta 32.2

Pro gammu = 3.0